In [7]:
import requests 
from urllib.parse import urljoin 
from urllib.parse import urlparse
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup 

In [3]:
sitemap_locations = [  '/sitemap.xml',
        '/sitemap_index.xml',
        '/wp-sitemap.xml',
        '/robots.txt' ]

url = "https://www.sandipuniversity.edu.in"
url2 = "https://www.anthropic.com"
response = requests.get(urljoin(url , "sitemap.xml")).text 

In [4]:
response 
response.split("\r\n")

['<?xml version="1.0" encoding="UTF-8"?>',
 '<urlset',
 '      xmlns="http://www.sitemaps.org/schemas/sitemap/0.9"',
 '      xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"',
 '      xsi:schemaLocation="http://www.sitemaps.org/schemas/sitemap/0.9',
 '            http://www.sitemaps.org/schemas/sitemap/0.9/sitemap.xsd">',
 '<!-- created with Free Online Sitemap Generator www.xml-sitemaps.com -->',
 '',
 '\n<url>\n  <loc>https://www.sandipuniversity.edu.in/</loc>\n  <lastmod>2025-02-07T09:18:47+00:00</lastmod>\n  <priority>1.00</priority>\n</url>\n<url>\n  <loc>https://www.sandipuniversity.edu.in/brochure_download.php</loc>\n  <lastmod>2025-02-07T09:18:47+00:00</lastmod>\n  <priority>0.80</priority>\n</url>\n<url>\n  <loc>https://www.sandipuniversity.edu.in/jobs/jobs.php</loc>\n  <lastmod>2025-02-07T09:18:47+00:00</lastmod>\n  <priority>0.80</priority>\n</url>\n<url>\n  <loc>https://www.sandipuniversity.edu.in/loan-facility.php</loc>\n  <lastmod>2025-02-07T09:18:47+00:00</lastmod>\

In [25]:
#############Href testinng################ 

response = requests.get(url).text 
soup = BeautifulSoup(response, 'html.parser') 
# links = soup.find_all(['a' , 'link'] , href=True)  
src_links  = soup.find_all( 'script' , src=True) 

links = extract_links_from_divs(soup , url)  

In [28]:
links

{'https://www.sandipuniversity.edu.in',
 'https://www.sandipuniversity.edu.in/',
 'https://www.sandipuniversity.edu.in/360/',
 'https://www.sandipuniversity.edu.in/4041.php',
 'https://www.sandipuniversity.edu.in/aboutus.php',
 'https://www.sandipuniversity.edu.in/academics',
 'https://www.sandipuniversity.edu.in/achievements/achievements.php',
 'https://www.sandipuniversity.edu.in/acm.php',
 'https://www.sandipuniversity.edu.in/admission-faq.php',
 'https://www.sandipuniversity.edu.in/admission-procedure.php',
 'https://www.sandipuniversity.edu.in/admission.php',
 'https://www.sandipuniversity.edu.in/alumni/about-alumni-cell.php',
 'https://www.sandipuniversity.edu.in/approvals.php',
 'https://www.sandipuniversity.edu.in/blog/',
 'https://www.sandipuniversity.edu.in/blog/events/',
 'https://www.sandipuniversity.edu.in/bom.php',
 'https://www.sandipuniversity.edu.in/brochure_download.php',
 'https://www.sandipuniversity.edu.in/chairman-word.php',
 'https://www.sandipuniversity.edu.in/c

In [ ]:
def extract_links_from_divs(soup, base_url):
    """Extract links from specific div elements"""
    links = set()
    # Find all div elements that might contain navigation/menu items
    target_divs = soup.find_all('div', class_=[
        'menu', 'nav', 'navigation', 'sidebar',
        'header', 'footer', 'main-menu', 'sub-menu',
        'navbar', 'top-menu', 'side-menu', 'main-nav',
        'sitemap', 'content-menu'
    ])
    # Also look for common navigation elements
    target_elements = soup.find_all(['nav', 'header', 'footer', 'aside'])
    # Process all target elements
    for element in target_divs + target_elements:
        # Find all anchor tags within the element
        for link in element.find_all('a', href=True):
            href = link.get('href')
            if href:
                # Clean and normalize the URL
                full_url = urljoin(base_url, href)
                # Only add if it's from the same domain
                if base_url in full_url and not any(
                    exclude in full_url.lower() for exclude in [
                        '#', 'javascript:', 'tel:', 'mailto:', 
                        'whatsapp:', 'facebook.com', 'twitter.com',
                        'linkedin.com', 'instagram.com'
                    ]
                ):
                    links.add(full_url)
    
    return links





In [ ]:
sitemaps = """/sitemap-index.xml/sitemap.php/sitemap.txt/sitemap.xml.gz/sitemap//sitemap/sitemap.xml/sitemapindex.xml/sitemap/index.xml/sitemap1.xml"""

possible_sitemaps = [sitemap for sitemap in sitemaps.split("/") if sitemap.strip()]  
possible_sitemaps 
    
    

['sitemap-index.xml',
 'sitemap.php',
 'sitemap.txt',
 'sitemap.xml.gz',
 'sitemap',
 'sitemap',
 'sitemap.xml',
 'sitemapindex.xml',
 'sitemap',
 'index.xml',
 'sitemap1.xml']

In [89]:
##########json-page-extraction 
from bs4 import BeautifulSoup 
ignore_urls = ["login", "signup"]



def get_domain(link) : 
    try :
        url_  = urlparse(link) 
        domain = url_.netloc.replace("www/" , " ") 
        return domain  
    except Exception as e : 
        print("error : " , e ) 
        raise e  
    
def isDomain(link : str , href : str ) : 
    try : 
        base_domain = get_domain(link) 
        href_domain = get_domain(href)
    
        return base_domain == href_domain ;  
    except :    
        return False  



def Extract_sites(base_link : str ) : 
    try :
        hrefs = set()
        response  = requests.get(base_link) 
        if response.status_code == 200:
            soup = BeautifulSoup(response.text , "html.parser") 
            for link in soup.find_all("a"):
                href = link.get("href") 
                # print(href)
                
                if href and href.startswith("https:") and not href.startswith("#")  and not href.startswith("tel:"):
                    
                    if not any(url in href.lower() for url in ignore_urls) and isDomain(base_link , href=href):
                        hrefs.add(href) 
        return hrefs                 
    except Exception as e : 
        print("error : " , e )
        raise e                         
            
            
        

In [90]:
sites = Extract_sites("https://www.sandipuniversity.edu.in") 

In [ ]:
sites

In [88]:
get_domain("https://in.linkedin.com/school/sandipuniversity/")

'in.linkedin.com'

In [ ]:
sitemap_locations = ['/sitemap.xml',
        '/sitemap_index.xml',
        '/wp-sitemap.xml',
        '/robots.txt']



for line in response.split("\r\n"):
    if "sitemap:" in line.lower(): 
        if not "sitemap.xml" in line.lower() : 
            continue   
        sitemap_url = line.split(": ")[1].strip()
        sitemap_locations.insert(0 , sitemap_url)   
        
 

def extract_sitemap(base_link : str ) : 
    try : 
        site_map = [] 
        for sites in sitemap_locations:
            response = requests.get(urljoin(base_link , sites)) 
            if response.status_code == 200:
                site_map.append(response.text)
                return site_map 
        url = urljoin(base_link , "robots.txt") 
        response = requests.get(url) 
        if response.status_code == 200: 
            for line in response.split("\r\n") : 
                if "sitemap:" in line.lower():
                    if not "sitemap.xml" in line.lower():
                        continue 
                site_map.append(line.split(": ")[1].strip())
    except Exception as e : 
        print("error : " , e )  
        raise e                 

SyntaxError: invalid syntax (2580626767.py, line 30)

In [20]:
titled_links = [{'href': 'https://docs.crawl4ai.com/', 'text': 'Crawl4AI Documentation (v0.4.3bx)', 'title': 'crawl', 'base_domain': 'crawl4ai.com'}, {'href': 'https://docs.crawl4ai.com/core/quickstart/', 'text': '', 'title': '', 'base_domain': 'crawl4ai.com'}] 

In [21]:
links = [titled_links[i].get("href") for i in range(len(titled_links))] 
valid_links = []
for url in titled_links : 
    if url.get("text") and url.get("text").strip():  
        links_info  = {
            "href" : url.get("href") , 
            "text" : url.get("text")
        } 
        if url.get("title") : 
            links_info["title"] = url.get("title")          
        valid_links.append(links_info)
valid_links         

[{'href': 'https://docs.crawl4ai.com/',
  'text': 'Crawl4AI Documentation (v0.4.3bx)',
  'title': 'crawl'}]

In [2]:
from urllib.parse import urlparse

In [5]:
url = "https://www.sandipuniversity.edu.in/result/results.php"

url_lower = url.lower() 
parsed_url = urlparse(url_lower) 
path = parsed_url.path.lower() 
path_segments = [s for s in path.split("/") if s.strip()]   
path_segments 

['result', 'results.php']